# Image Processing Pipeline **Setup**
--------------

## **OVERVIEW**

The first thing we need to be able to do is access the image files and interact with them (e.g., read the metadata).

#### &#x1F4D6; **How to:** 

Advance through each block of code by pressing `Shift`+`Enter`.

## **IMPORTS**

The convention with notebooks (and python in general) is to import the nescessary packages as the first thing.


We are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.

> ###### 📝 **There are a few convences used here worth explanation.  Note the `imports.py` and `constants.py` files in the base level of the `infer_subc` module.  These provide shortcuts for keeping track of imports and constants.  cf. the bottom of the imports below.  A second thing to note is the use of the "magics" `%load_ext autoreload` `%autoreload 2`, which tells the notebook to reload any changes made in the source code of the module on change; hence, avoid re-executing the imports.**


#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block loads all of the necessary python packages and functions you will need for this notebook. 

In [ ]:
# top level imports
from pathlib import Path
import os

import napari
import pandas as pd

from infer_subc.core.file_io import (read_czi_image, list_image_files)

%load_ext autoreload
%autoreload 2

## **LOAD AND READ IN IMAGE FOR PROCESSING**
Read the image and metadata into memory as an ndarray and dictionary from the `.czi` or '.tiff' files.

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
# Specify the file type of your raw data that will be analyzed. Ex) ".czi" or ".tiff"
im_type = ".czi"

# Specify the folders in which your data is located:
## Define the path to the directory that contains the input image folder.
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"

## Specify which subfolder that contains the input data and the input data file extension
in_data_path = data_root_path / "raw"

## Specify the output folder to save the segmentation outputs if.
## If its not already created, the code below will creat it for you
out_data_path = data_root_path / "out"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
img_file_list = list_image_files(in_data_path,im_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":img_file_list})

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
# Based on the output table above, select the file you'd like to examine.
test_img_n = 5

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

print("Metadata information")
print(f"Channel names: {channel_names}")
print(f"Image name: {img}")
print(f"Scale: {scale}")
print(f"Channel axis: {channel_axis}")

# Visualize the image in the Napari window
viewer = napari.Viewer()
viewer.add_image(img_data)
print("\nProceed to Napari window to view your selected image.")

--------------
### NEXT:  INFER MASKS

proceed to:

[**Default Workflow (1)**](./1.1_infer_masks_from-composite_with_nuc.ipynb) - for images with cytoplasmic organelles, a **nuclei** marker, no **cell membrane** makers, and more than one cell per field of view

[**Alternative Workflow (A)**](./1.1a_infer_masks_from-composite_single_cell.ipynb) - an alternative workflow for images with only cytoplasmic organelles, NO **nuclei** or **cell membrane** makers, one cell per field of view

[**Alternative Workflow (B)**](./1.1b_infer_masks_from-composite_multiple-cells.ipynb) - an alternative workflow for images with only cytoplasmic organelles, NO **nuclei** or **cell membrane** makers, and more than one cell per field of view
